# Faceted search with postgres using tsvectors

<b>TL;DR</b> Faceting adds a rich layer of interaction into underlying data, however when the data starts getting big, serving a faceted-search operation in real-time poses a computational challenge. Using a distributed search engine such as Elasticsearch and parallelizing aggregation operations has become the way to go to cut the response times. In this post, we will discuss how you can use a lightweight single-instance postgres setup with its text-search-vector (tsvector) data type to enable faceted-search over medium-sized datasets avoiding the need to setup a Elasticsearch cluster or a similar infrastructure for that scale.

## What is faceted search?

A __facet__ can be defined as a curated division of a dataset with an intent to study, explore or search items within it. And thus __faceted-search__ is a search operation facilitated via facets to help narrow down search context based on a criteria of interest. 

From real-world examples, facets are created via categories (or subcategories) that you'd see in search results of an application. Fig 1. shows an example of facets (and their counts) with respect to `Listing Type` of houses in [zillow](https://www.zillow.com/). When you set filter conditions on number of bedrooms or price-range of the houses, facet-counts get updated and thus gives you an informational summary of the results that you are about to see.

The term faceted-search gained popularity with advent and evolution of powerful search frameworks such as [Apache's lucene](https://lucene.apache.org/). Lucene's documentation also includes a [well-explained document](http://lucene.apache.org/core/4_0_0/facet/org/apache/lucene/facet/doc-files/userguide.html) on different aspects of faceted-search.

<img src="img/facets-example.jpg" alt="drawing" width="400" />

## Why discuss postgres for faceted-search?
Sophisticated search-platforms such as Solr and Elasticsearch make a highly performant faceted-search accessible to any application. Both Solr and Elasticsearch use [Lucene's inverted index](https://lucene.apache.org/core/5_1_0/core/org/apache/lucene/codecs/lucene50/package-summary.html#Overview) to make term-based search efficient. Search performance is then further enhanced by distributing the data in a cluster and storing replicas. Yet, a lot of typical web-applications rely on using a SQL-based relational database such as postgres for data storage and search needs because it is very lightweight to setup in production with requirements such as access-control, versioning schema-migrations and simple integration with a given front-end technology.  

However, the lightweight setup plays against postgres when it comes to supporting complex operations on large datasets such as ... _faceted-search_. Postgres inherently doesn't support a distributed storage or a distributed query engine to make use of a multi-node setup. And with this, faceted-search operation on a large dataset could be extremely slow. We will show there are optimizations possible beyond an optimized search index to make faceting way more efficient by leveraging the functionality of postgresql's [text-search vectors](https://www.postgresql.org/docs/8.3/datatype-textsearch.html) (tsvectors). However, the extent of these optimizations, as we will see later, is limited and applicable only for upto medium-sized datasets (in order of 100k records with 15-50 metafields).


Before we get to concretely define the problem we are trying to solve, one thing to be noted is that the search aspect of _faceted-search_ is a plain-old filter operation, complex part is showing rich useful facets. And thus, our focus here would be in building rich dynamic facets which get updated each time user changes filter conditions.

## Problem Definition

Assume that we are working on a dataset of patient-health records where-in each record has certain metadata associated to it as a set of meta_field-value pairs. For example, `patient's state` is a meta_field and `California`, `Arizona`, etc. are its associated values.
For practical use-cases, it is fair to assume total number of unique meta_field-value pairs can be fairly diverse (~100K) but number of possible meta_fields is small (<100).

        record - [ set of (meta_field, value) pairs ]

Lets define `filter` condition on a record as below:

        filter - [ set of (meta_field, value) pairs expected in a record's metadata ]

A `faceted-search` operation on this dataset would rely upon using meta_field as the source of facet (a category).
More concretely, we will define `faceted-search` operation as below:

#### faceted-search pseudo-code

    with filter = <user-defined filter>
    for each meta_field in dataset:
      find top-k most frequently occuring values of the meta_field wrt # of records
      s.t only records which meet `filter` condition are counted

The above operation does look a bit complex irrespective of what data model you choose because you are doing a filtered-aggregation operation over entire dataset for each meta-field.

### Traditional SQL solution

For considering a SQL solution, we have to define a data-model representation. Applications are usually constrained by their own schema requirements and a generic data-model mighn't work for all use cases. So without taking into account any application-specific schema constraints, for a best-case scenario, lets assume an efficient collapsed data model mapping each record to its metadata as below:

<div id='tab1' style='float:left'>
    <table>
      <tr>
        <th></th>
        <th>Data Type</th>
        <th style="text-align:left">Description</th>          
      </tr>
      <tr>
        <td><b>record_id</b></td>
        <td>UUID</td>
        <td style="text-align:left">Unique identifier to the record</td>          
      </tr>
      <tr>
        <td><b>metadata</b></td>
        <td>jsonb</td>
        <td style="text-align:left">Binary json object mapping each meta_field to its value</td>
      </tr>
    </table>
</div>

#####  Representative SQL-query for a meta-field `state`

For illustrative purposes, here's what a SQL query resolving the above pseudocode for a meta_field `state` in record's `metadata` would look like:

    SELECT
     records.metadata -> 'state' AS meta_field,
     count(*) AS meta_field_count
    FROM
     records
    WHERE
     records.metadata @> 
         '{"patient_ethnicity": "Caucasian",
           "provider_specialty_string": "Acute Care"}'::jsonb
    GROUP BY
     meta_field
    ORDER BY
     meta_field_count
    LIMIT
     50

Here, `facet` in action is `Caucasian` patients treated by `Acute Care` specialists. 

`@>` is an efficient [jsonb operator](https://www.postgresql.org/docs/9.6/functions-json.html#FUNCTIONS-JSONB-OP-TABLE) that can be indexed to efficiently check for containment of one jsonb record with respect to another.

You would repeat the above operation for all meta-fields to meet requirements of faceted-search.

While we do have a working solution in hand with simple readable code but in practice, performance for the above query is sub-par and as we would demonstrate later in the _Evaluation_ section, it can take near about 10 seconds to get a response for a dataset of size 100k records with 10+ meta-fields. Any performance improvements here would greatly benefit user experience. We will show next how you can use text-search features in postgres to get multi-fold performance gains.

## Redefining problem using tsvectors

### Text-search in postgres and tsvectors

When full text-search got introduced in Postgresql, it introduced two new datatypes, [tsvector](https://www.postgresql.org/docs/9.6/datatype-textsearch.html#DATATYPE-TSVECTOR) for storing preprocessed documents and, [tsquery](https://www.postgresql.org/docs/9.6/datatype-textsearch.html#DATATYPE-TSQUERY) for textual queries. `tsvector` is defined as sorted list of unique lexemes and makes text-search work faster than the `LIKE` operator, while `tsquery` is an expression on lexemes with boolean operators between them. You can convert text and arrays to tsvectors and can further do other specific operations on tsvectors, most prominently, text search. With `tsvector`, `tsquery` and full-text-search (FTS) operator `@@`, you can determine if certain combination of words can be found in a text document.

    =# select 'The patients show symptoms of cough along 
                 with fever'::tsvector @@ 'cough & fever'::tsquery;
    ?column?
    ----------
    t

The above sql statement tests if words `cough` and `fever` both appear (`&` operator) in the sentence `The patients show symptoms of cough along with fever`. When, a text sentence is cast to `tsvector`, lexemes are extracted out to build a sparse representation of the sentence on which a text-query can be run. 

### Using tsvectors to redefine faceted-search

The complexity of faceted-search problem defined above amounts to looking at the frequency of occurence of all qualified (meta_field, value) pairs. Thus, if we model each (meta_field, value) pair as a word instead, this becomes a word-count problem. And guess what, we don't have to implement a word-count function, its inbuilt ([ts_stat](https://www.postgresql.org/docs/9.6/functions-textsearch.html)) into postgres as it obliges itself as a text-search engine. 

Building onto this, a `record` becomes a set of words and thus can be represented as a text document. Group of records combined together is still a text document. And same applies to `filter` too, however, as we move towards a text-search problem, we can call `filter` as a text-search query (__ts_query__) on the text-search vector (__tsvector__) of `record`. 

Thus, the problem can now be defined as:

    with filter = <user-defined filter>

    get word counts over all such `record` tsvectors
       which match `filter` tsquery

    process words from word-counts to get top-k values per meta-field

### Data model

To implement this approach, we need a model that looks like below:

<div id='tab2' style='float:left'>
    <table>
      <tr>
        <th></th>
        <th>Data Type</th>
        <th style="text-align:left">Description</th>
      </tr>
      <tr>
        <td><b>record_id</b></td>
        <td>UUID</td>
        <td style="text-align:left">Unique identifier to the record</td>
      </tr>
      <tr>
        <td><b>record_tsv</b></td>
        <td>tsvector</td>
        <td style="text-align:left">text-search vector representation of (meta-field, value) pairs encoded as words</td>          
      </tr>
    </table>
</div>

We encode each (meta-field, value) pair in a record as a word and combine words together to form a document. Finally, we turn this document to a text-search vector.

<b>NOTE:</b> Even if you are bound by application schema constraints, you can build a [materialized view](https://www.postgresql.org/docs/9.6/rules-materializedviews.html) from your tables so that the data representation minimally has above columns. 

#####  Representative SQL-query for all meta-fields combined

    select meta_field, value, ndoc as freq
    /* Parse result of word-counts to get top-50 values
    for each `meta_field` by frequency of occurence */    
    from                                              
      (                                       
        select
          -- Break encoded words to get `meta_field`
          split_part(word, ':', 1) as meta_field,    
          split_part(word, ':', 2) as value,
          ndoc,
          -- Rank each `value` of a `meta_field` by frequency of occurence
          row_number() OVER (                         
            partition by split_part(word, ':', 1)
            order by ndoc desc
          ) as rank
        from
          -- Word-count on all qualified records       
          ts_stat(                                   
            $$
            select
              record_tsv
            from
              records
              -- Filter records based on the given criteria
              where record_tsv @@ to_tsquery(              
                  'provider_specialty_string:Multispecialty' & 
                  'patient_ethnicity:Caucasian')    
       $$
          )
      ) meta_field_value_counts
    where
      rank <= 50;

The query does look a little bit complicated probably because `ts_stat` expects a sql query as an argument that should return a list of `tsvectors`. Word counts are computed upon these `tsvectors` and then you parse results to extract field and value from the words. But the performance gains we get out of this is significant. We observed 4x-5x gains on query response times (elaborated in _Evaluation_ section below) as compared to the traditional sql approach discussed above.

Here, preprocessing is required to convert each record's metadata to a `tsvector`. You can see `to_tsquery` function call above as to how we formed a word `patient_ethnicity:Caucasian` using `("patient_ethnicity", "Caucasian")` as (meta_field, value) pair. If you have special characters in your meta-field and/or value, its recommended to build tsvector by first building an array of words and then using [array_to_tsvector](https://www.postgresql.org/docs/current/functions-textsearch.html#TEXTSEARCH-FUNCTIONS-TABLE) to construct the `tsvector` to ensure that lexemes extracted from word are identical to the word itself (and not some reduced form of word).

## Evaluation

Before we do performance measurements and comparisons, one thing to be noted is that query performance for all approaches depends on the coverage of selected facet (`filters`). That is, query performance for a facet that covers just 0.1% of dataset will be vastly better than a facet that covers 50% of the same dataset. 

### Comparison with traditional-sql approach

Starting with that note, lets see how tsvectors approach compares with tradition-sql approach used above on a 100K dataset with 15 meta-fields on each record. To optimize performance for traditional-sql approach, we used a [GIN index](https://www.postgresql.org/docs/9.4/datatype-json.html#JSON-INDEXING) on `metadata` column with `jsonb_path_ops` as operator class. For tsvectors approach, we used [GIN index](https://www.postgresql.org/docs/9.6/textsearch-indexes.html) for text-search on `record_tsv` column.

<img src="img/traditional-vs-tsvectors100k.png" alt="drawing" width="700" />

As coverage of facet increases, `tsvectors` approach edges out significantly against traditional-sql approach. Facet coverage here is indicative of the number of records that are worked on by the group-by operation in tradtional-sql appraoch and `ts_stat` operation in tsvector approach. Clearly, `ts_stat` owns the battle.

**NOTE**: We use CPU time and not wall-time to measure performance as you can set up concurrent SQL threads (one for each meta-field query) to make the operation return results quickly but database's resources are unavailable for other queries for that time.

### Comparison with Elasticsearch

Given that tsvectors obtains significant performance gains over traditional-sql approach, the question that remains to be answered is - can it replace a distributed lucene-powered search engine such as Elasticsearch.

Lets see how performance of tsvectors approach on a single instance postgres database compares with a 3-node Elasticsearch cluster. Also, lets bump the stakes as well and use a dataset sized with 1 million records and ~80 field-value pairs per record on average.

Here are the numbers:

<img src="img/tsvectors-vs-Elasticsearch-1M.png" alt="drawing" width="700" />

As you can see, we have used wall times (and not CPU-time) to measure performance unlike the previous case where we were trying to estimate performance over a unit CPU resource. This is because ES-clusters aren't resource bound and can be scaled as per needs and thus wall-time can be optimized. 

From the results above, we can see that response times for the postgres query can go upto 70 seconds. Usually web-applications have a response timeout when a query takes more than a minute. This just renders postgres unusable for faceted-search on datasets of size 1M or more. Elasticsearch however can be scaled with as many number of nodes you would want (in theory) and can thus overcome performance limitations with huge datasets.

## Conclusion

We introduced the technical description of faceted-search problem and then described how you could best handle it with a traditional SQL solution of grouping records based on a meta-field after filtering them based on a given criteria. Having learnt about tsvectors, we then approached the problem using text-search features of postgres and found them to be immensely efficient when evaluating the performance on a dataset with a hundred thousand records and comparing it with traditional SQL solution. 

However, when the dataset size was increased ten-fold to a million records instead, we found execution times to be nearly impractical for production use. Switching to Elasticsearch makes the problem much more manageable by reducing the execution time to near real-time. It sets us up for a conclusion that `tsvectors` are fast and efficent for faceted-search if you are predominantly working with small-sized datasets (in order of 100k records) or if you want to just try out faceted-search in your postgres-powered web-application without having to stand up a new infrastructure. 

If you anticipate the scale of datasets employed for faceted-search to extend in the order of millions, you will have to prepare yourselves for the move to Elasticsearch or a similar distributed-search framework. Tools such as [logstash](https://www.elastic.co/blog/logstash-jdbc-input-plugin) make it easy to migrate data but application-code and the data-platform setup becomes the greatest liability when making the move. Using a logical data layer between application and data-storage can curtail application-code related setbacks to minimum and help in a smooth transition.